In [1]:
!pip install anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.2/388.2 kB 6.6 MB/s eta 0:00:00


In [6]:
import anthropic
from anthropic.types import MessageParam, ToolUnionParam
from google.colab import userdata

## Tools

In [3]:
add_tool = {
    "name": "add",
    "description": "Add two numbers together. Use this tool for addition operations.",
    "input_schema": {
        "type": "object",
        "properties": {
            "a": {"type": "number", "description": "The first number to add."},
            "b": {"type": "number", "description": "The second number to add."},
        },
        "required": ["a", "b"],
    },
}

multiply_tool = {
    "name": "multiply",
    "description": "Multiply two numbers together. Use this tool for multiplication operations.",
    "input_schema": {
        "type": "object",
        "properties": {
            "a": {"type": "number", "description": "The first number to multiply."},
            "b": {"type": "number", "description": "The second number to multiply."},
        },
        "required": ["a", "b"],
    },
}

power_tool = {
    "name": "power",
    "description": "Raise a number to a power (exponentiation). Use this tool for power/exponentiation operations, e.g. 2^3, 2 to the power of 3, etc. "
    + "Usage example: to calculate 2^3, use this tool with base=2 and power=3.",
    "input_schema": {
        "type": "object",
        "properties": {
            "base": {
                "type": "number",
                "description": "The base number to raise to a power.",
            },
            "power": {"type": "number", "description": "The exponent/power."},
        },
        "required": ["base", "power"],
    },
}


def add(a: float, b: float) -> str:
    return f"{a} + {b} = {a + b}"


def multiply(a: float, b: float) -> float:
    return f"{a} * {b} = {a * b}"


def power(base: float, power: float) -> str:
    return f"{base} ^ {power} = {base ** power}"

## Tool call dispatcher

In [4]:
tool_dict = {
    add_tool["name"]: (add_tool, add),
    multiply_tool["name"]: (multiply_tool, multiply),
    power_tool["name"]: (power_tool, power),
}


def process_tool_call(tool_name, tool_input) -> str:
    if tool_name not in tool_dict:
        return f"Error: Unknown tool name: {tool_name}."
    tool_schema, tool_func = tool_dict[tool_name]
    try:
        valid_keys = tool_schema["input_schema"].get("properties", {}).keys()
        filtered_input = {k: v for k, v in tool_input.items() if k in valid_keys}
        return tool_func(**filtered_input)
    except Exception as e:
        return f"Error: {e}"

## Agent loop

In [ ]:
# ANSI color codes
CYAN = "\033[96m"
YELLOW = "\033[93m"
GREEN = "\033[92m"
RED = "\033[91m"
RESET = "\033[0m"

In [7]:
def run_agent(
    user_query: str,
    system_prompt: str,
    client: anthropic.Anthropic,
    model_name: str = "claude-3-haiku-20240307",
    tools: list[ToolUnionParam] = [],
    max_iterations: int = 10,
) -> tuple[str | None, list[MessageParam]]:
    messages = [{"role": "user", "content": user_query}]
    current_iteration = 0

    while current_iteration < max_iterations:
        current_iteration += 1

        print(f"***LOG: sending messages: {messages}")
        response = client.messages.create(
            max_tokens=1024,
            model=model_name,
            system=system_prompt,
            messages=messages,
            tools=tools,
        )
        print(f"***LOG: response: {response}")
        print("-" * 20)
        messages.append({"role": "assistant", "content": response.content})

        if response.stop_reason == "tool_use":
            tools_calls = [
                block for block in response.content if block.type == "tool_use"
            ]
            tool_results = []
            for tool_call in tools_calls:
                tool_name = tool_call.name
                tool_input = tool_call.input
                output = process_tool_call(tool_name, tool_input)
                print(f"***LOG: tool call: {tool_call.name} with input: {tool_input} returned: {output}")
                tool_results.append(
                    {
                        "type": "tool_result",
                        "tool_use_id": tool_call.id,
                        "content": output,
                    }
                )
            messages.append({"role": "user", "content": tool_results})

        elif response.stop_reason == "end_turn":
            break
        else:
            print(f"WARNING: unexpected stop reason: {response.stop_reason}")

    if current_iteration >= max_iterations:
        print(f"WARNING: Max iterations reached. Returning partial response.")

    final_content = next(
        (content.text for content in response.content if content.type == "text"),
        None,
    )
    return final_content, messages


In [8]:
def test_calculator_agent(user_query):
    api_key = userdata.get("ANTHROPIC_API_KEY")
    client = anthropic.Anthropic(api_key=api_key)

    tool_names = ", ".join(tool_dict.keys())

    system_prompt = f"""
You are a calculator agent with access to the following tools: {tool_names}.
- ONLY use the tools you have access to. DO NOT attemp to use any other tools not listed here.
- For simple calculation, directly invoke the appropriate tool corresponding to the operation you need to perform.
- For complex calculations, break down the problem into multiple steps and invoke tools accordingly.
  1. first calculate all independent sub-expressions in parallel.
  2. then combine the results
- Invoke the tools sequentially if the result of the previous tool call is needed for the next tool call.
Example: to calculate (2 + 3) * 4,
  1. first invoke the add tool with a=2 and b=3,
  2. then invoke the multiply tool with a=result of add tool and b=4.
- When multiple operations can be calculated indenpendently, invoke all the tools needed simultaneously in a single response.
Example: to calculate (1+2) * (3+4),
  1. first, notice that 1+2 and 3+4 can be calculated independently.
  Therefore, perform 2 simultaneous invocations of the add tool, i.e. call both add(1, 2) and add(3, 4) in a single response.
  2. then invoke the multiply tool with a=[result of add(1, 2)] and b=[result of add(3, 4)].
- When multiple INDEPENDENT calculations are needed, call tools in parallel in a single response.
- Once you have the final result, explain the calculation step by step in the final response.
    """
    print(f"***LOG: system prompt: {system_prompt}")

    tools = [tool_schema for tool_schema, _ in tool_dict.values()]

    final_content, messages = run_agent(
        user_query,
        system_prompt,
        client,
        model_name="claude-3-haiku-20240307",
        tools=tools,
        max_iterations=10,
    )

    if final_content is None:
        print("WARNING: Claude returned no content.")
    else:
        print("Claude's response:", final_content)

    print("--------Message history--------")
    for i, message in enumerate(messages):
        print(f"[{i}] {message['role']}: {message['content']}")
    print("--------End of message history--------")

## Tests

In [9]:
test_calculator_agent("What's 123+456?")

***LOG: system prompt: 
You are a calculator agent with access to the following tools: add, multiply, power.
- ONLY use the tools you have access to. DO NOT attemp to use any other tools not listed here.
- For simple calculation, directly invoke the appropriate tool corresponding to the operation you need to perform.
- For complex calculations, break down the problem into multiple steps and invoke tools accordingly.
  1. first calculate all independent sub-expressions in parallel.
  2. then combine the results
- Invoke the tools sequentially if the result of the previous tool call is needed for the next tool call.
Example: to calculate (2 + 3) * 4,
  1. first invoke the add tool with a=2 and b=3,
  2. then invoke the multiply tool with a=result of add tool and b=4.
- When multiple operations can be calculated indenpendently, invoke all the tools needed simultaneously in a single response. 
Example: to calculate (1+2) * (3+4), 
  1. first, notice that 1+2 and 3+4 can be calculated indep

In [10]:
test_calculator_agent("calculate 12 x 34")

***LOG: system prompt: 
You are a calculator agent with access to the following tools: add, multiply, power.
- ONLY use the tools you have access to. DO NOT attemp to use any other tools not listed here.
- For simple calculation, directly invoke the appropriate tool corresponding to the operation you need to perform.
- For complex calculations, break down the problem into multiple steps and invoke tools accordingly.
  1. first calculate all independent sub-expressions in parallel.
  2. then combine the results
- Invoke the tools sequentially if the result of the previous tool call is needed for the next tool call.
Example: to calculate (2 + 3) * 4,
  1. first invoke the add tool with a=2 and b=3,
  2. then invoke the multiply tool with a=result of add tool and b=4.
- When multiple operations can be calculated indenpendently, invoke all the tools needed simultaneously in a single response. 
Example: to calculate (1+2) * (3+4), 
  1. first, notice that 1+2 and 3+4 can be calculated indep

In [11]:
test_calculator_agent(" What's 2 to the power of 10?")

***LOG: system prompt: 
You are a calculator agent with access to the following tools: add, multiply, power.
- ONLY use the tools you have access to. DO NOT attemp to use any other tools not listed here.
- For simple calculation, directly invoke the appropriate tool corresponding to the operation you need to perform.
- For complex calculations, break down the problem into multiple steps and invoke tools accordingly.
  1. first calculate all independent sub-expressions in parallel.
  2. then combine the results
- Invoke the tools sequentially if the result of the previous tool call is needed for the next tool call.
Example: to calculate (2 + 3) * 4,
  1. first invoke the add tool with a=2 and b=3,
  2. then invoke the multiply tool with a=result of add tool and b=4.
- When multiple operations can be calculated indenpendently, invoke all the tools needed simultaneously in a single response. 
Example: to calculate (1+2) * (3+4), 
  1. first, notice that 1+2 and 3+4 can be calculated indep

In [12]:
test_calculator_agent("Calculate (5 + 3) × 2")

***LOG: system prompt: 
You are a calculator agent with access to the following tools: add, multiply, power.
- ONLY use the tools you have access to. DO NOT attemp to use any other tools not listed here.
- For simple calculation, directly invoke the appropriate tool corresponding to the operation you need to perform.
- For complex calculations, break down the problem into multiple steps and invoke tools accordingly.
  1. first calculate all independent sub-expressions in parallel.
  2. then combine the results
- Invoke the tools sequentially if the result of the previous tool call is needed for the next tool call.
Example: to calculate (2 + 3) * 4,
  1. first invoke the add tool with a=2 and b=3,
  2. then invoke the multiply tool with a=result of add tool and b=4.
- When multiple operations can be calculated indenpendently, invoke all the tools needed simultaneously in a single response. 
Example: to calculate (1+2) * (3+4), 
  1. first, notice that 1+2 and 3+4 can be calculated indep

In [13]:
test_calculator_agent("What's 2^3 and 4^2?")

***LOG: system prompt: 
You are a calculator agent with access to the following tools: add, multiply, power.
- ONLY use the tools you have access to. DO NOT attemp to use any other tools not listed here.
- For simple calculation, directly invoke the appropriate tool corresponding to the operation you need to perform.
- For complex calculations, break down the problem into multiple steps and invoke tools accordingly.
  1. first calculate all independent sub-expressions in parallel.
  2. then combine the results
- Invoke the tools sequentially if the result of the previous tool call is needed for the next tool call.
Example: to calculate (2 + 3) * 4,
  1. first invoke the add tool with a=2 and b=3,
  2. then invoke the multiply tool with a=result of add tool and b=4.
- When multiple operations can be calculated indenpendently, invoke all the tools needed simultaneously in a single response. 
Example: to calculate (1+2) * (3+4), 
  1. first, notice that 1+2 and 3+4 can be calculated indep